In [ ]:
!pip install openai

In [ ]:
!pip install mysql-connector-python

In [ ]:
# package

import openai
import mysql.connector
from pprint import pprint # chatGPT message가 잘 출력되되도록 사용
import pandas as pd
import numpy as np
from datetime import datetime
from pathlib import Path

In [ ]:
# Google Drive 마운트
# 마운트는 본인 드라이브만 가능하기에 공유 폴더 같은 경우 본인 드라이브에 바로가기를 추가하여 사용

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# API KEY : contact 계정의 API key 발급하여 사용

openai.api_key = ""

In [ ]:
today = datetime.today().strftime("%Y%m%d")
Path(f"/content/drive/MyDrive/HAI research/output/{today}/py_output").mkdir(parents=True, exist_ok=True)

In [ ]:
# google sheet를 바로 연동하는 것은 원본 훼손 위험이 존재하기에 우선 xlsx 파일을 다운로드 및 업로드하여여 활용

raw_data = pd.read_excel("/content/drive/MyDrive/HAI research/data/PoC 평가.xlsx", sheet_name='total')

In [ ]:
en_data = raw_data.copy()

In [ ]:
# PoC 평가가 A+인 인원들만 불러오기
# url 주소에 userID가 존재하는 형식 데이터만 불러오기
en_data = en_data.loc[en_data['총평 \n(Letter grade, like A+, B+)'] == 'A+']
en_data = en_data[en_data['framing docs'].str.contains('&userID', na = False)].reset_index(drop=True)

In [ ]:
# '&' 기준으로 파싱하여 user_id 리스트에 저장하기
id_list = []
for i in range(len(en_data)) :
  id_list.append(en_data['framing docs'][i].split('&')[1][7:])

In [ ]:
# DB 연동

mydb = mysql.connector.connect(
  host="",
  user="",
  password="",
  database=""
)

# 저장된 userID 정보 불러와서 사용
userCode = id_list[8]

mycursor = mydb.cursor()

mycursor.execute("SELECT user_id from userCodeTable where code='" + userCode + "'")

myresult = mycursor.fetchall()

userID = myresult[0][0]

mycursor.execute("SELECT user_id, question_id, response FROM responseTable where user_id='" + str(userID) + "'")

myresult = mycursor.fetchall()

userResponse = {}

# 0부터 시작되는 문항번호를 1부터 시작하도록 변경
for x in myresult:
    userResponse[x[1]+1] = x[2]

In [ ]:
userResponse

In [ ]:
# 문항 정보 입력
qna = pd.read_csv("/content/drive/MyDrive/HAI research/data/PoC_Question.csv")

In [ ]:
widget = ["Accuracy chart", "Bar chart", "Scatter chart"]
purpose = "임직원들의 승진 여부를 예측하는 것"

messages = [{"role": "system", "content": "너는 AI 분야의 data scientist야"},
            # {"role": "system", "content": "너는 인공지능 목적에 적합한 시각화 위젯을 3개만 추천해줘"},
            {"role" : "system", "content" : f"너는 시각화 위젯을 추천할 때, {widget}안에 존재하는 시각화 위젯만 반드시 추천해줘"},
            {"role" : "system", "content" : f"{widget[0]}은 시각화 plot에 해당하고 인공지능 모델의 정확도를 시각화하여 나타낸 그래프야"},
            {"role" : "user", "content" : f"인공지능의 목적은 {purpose}야. 해당 목적에 부합하는 시각화 위젯을 그래프 이름 및 설명과 함께 3개 추천해줘"}
            ]

completion = openai.ChatCompletion.create(
model="gpt-3.5-turbo",
messages=messages,
temperature=0
)
'''
chat_response = completion.choices[0].message.content

messages=[{"role" : "system", "content" : "너는 AI 분야의 data scientist야"},
          {"role" : "system", "content" : "너는 적합한 시각화 위젯을 추천했는지 리뷰하는 역할을 할거야"},
          {"role" : "system", "content" : f"{chat_response}가 {widget}안에 있는 시각화 그래프들을 추천해주지 않았을 경우, {widget}안에 있는 시각화 그래프들로 다시 3개 추천해줘"},
          {"role" : "user", "content" : f"{chat_response}를 리뷰할거야. 적절한 시각화 위젯을 추천했다면 해당 결과를 그대로 보여줘"}
          ]

completion = openai.ChatCompletion.create(
model="gpt-3.5-turbo",
messages=messages,
temperature=0
)
'''
chat_response = completion.choices[0].message.content
pprint(chat_response)